
**autor:** @SamyConejo


*   **Tema:** Prototipo de sistema móvil para reconocimiento facial aplicando Redes Neuronales Convolucionales y Transferencia de Aprendizaje.


***Descripción***
1.  Configuración de la RNC MobileNetV2 para aplicar Transfer Learning y exportar el modelo en formato .tflite para implementación en dispositivo móvil.




In [ ]:
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, classification_report
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers
from keras.models import load_model
import cv2
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from statistics import mean
import itertools

In [ ]:
# paths to dataset
base_dir = '/Users/samcn96/PycharmProjects/training/rostros/dataset'
train_data_dir='/Users/samcn96/PycharmProjects/training/rostros/dataset/training'
validation_data_dir='/Users/samcn96/PycharmProjects/training/rostros/dataset/validation'
test_path='/Users/samcn96/PycharmProjects/training/rostros/dataset/test'

In [ ]:

IMAGE_SIZE = 224 # image size
BATCH_SIZE = 64  # number of images we are inputting into the neural network at once.
EPOCHS = 100 # number of epochs to train
NUM_CLASSES = 10 # we have 10 people to predict

# scalling images to [0 - 1]
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
)

train_generator = datagen.flow_from_directory( # training generator
    train_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
    batch_size = BATCH_SIZE,
    shuffle = True
)
val_generator = datagen.flow_from_directory(  #validation generator
    validation_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle = True
)

In [ ]:
# save labes.txt to use later in App Development

print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)


In [ ]:
# MobileNetV2, not including top layers to enable Transfer Learning

MobileNet = MobileNetV2(weights='imagenet',
                      include_top=False,
                      input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# We freeze layers, except 8 last layers.
count = 0
for layer in MobileNet.layers:
    if count < 146:
        layer.trainable = False
    count += 1

# print layers to check freeze    
for (i, layer) in enumerate(MobileNet.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)
    


In [ ]:
# our top custom layers.
def lw(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(NUM_CLASSES,activation='softmax')(top_model)
    return top_model

FC_Head = lw(MobileNet, NUM_CLASSES)
model = Model(inputs = MobileNet.input, outputs = FC_Head)

# print our custom model summary
print(model.summary())


In [ ]:
# we use callbacks to control training process

checkpoint = ModelCheckpoint("face_recognized.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             period=1)

reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.5,
                              patience = 2,
                              verbose = 1,
                              mode = 'max',
                              min_lr = 0.000001)

earlystop = EarlyStopping(monitor = 'val_accuracy',
                          patience = 5,
                          verbose = 1,
                          mode = 'max',
                          restore_best_weights = True)


callbacks = [earlystop, reduce_lr, checkpoint]

In [ ]:
# compile and fit model with our configuration, we use lr = 10-4, Adam optimizer.

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs = EPOCHS,
    callbacks = callbacks,
    validation_data=val_generator
)

In [ ]:
# Training and Validation Accuracy

print('Mean Training Accuracy ', mean(history.history['accuracy']))
print('Mean Validation Acurracy ', mean(history.history['val_accuracy']))


In [ ]:
# save history
data = str(history.history)
with open('history.txt', 'w') as f:
    f.write(data)

In [ ]:
# now we plot performance of training and validation.

plt.plot(history.history['loss'], label = 'Training')
plt.plot(history.history['val_loss'], label = 'Validation')

plt.xlabel("Epoch")
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss.png')
plt.close()

plt.plot(history.history['accuracy'], label = 'Training')
plt.plot(history.history['val_accuracy'], label = 'Validation')


plt.xlabel("Epoch")
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.close()


In [ ]:
LABELS =["Elvis", "Fausto", "Ale", 'Jenifer', 'Malki', 'Maru', 'Nayta', 'Roberto','Samy', 'Sulay']

def my_metrics(y_true, y_pred):
  
    
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("F1 Score  : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
   
    report = classification_report(y_true, y_pred, labels=[0,1,2,3,4,5,6,7,8,9], 
                                   target_names=LABELS)
    print(report)
    return accuracy, precision, f1Score, cm

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
 
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.savefig('matrix.png')
    plt.close()

In [ ]:
# test model over test folder
print("==============TEST RESULTS============")
test_generator = datagen.flow_from_directory(
        test_path,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode = None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore, cm = my_metrics(true_classes, yPredictions)

plot_confusion_matrix(cm, LABELS)



In [ ]:
# we can export the model

saved_model_dir = 'mobile'
tf.saved_model.save(model, saved_model_dir) 

# we converte .h5 model to .tflite to mobile on device implementation.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert() 

with open('mobile/model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
# we have our model.h5 and model.tflite, and lets test on testing files.

# dic to map output of model inference and labels
my_face_dict =  { "[0]": "Elvis",
                  "[1]": "Fausto",
                  "[2]": "Ale",
                  "[3]": "Jenifer",
                  "[4]": "Malki",
                  "[5]": "Maru",
                  "[6]": "Nayta",
                  "[7]": "Roberto",
                  "[8]": "Samy",
                  "[9]": "Sulay",
                      }
# dic to print class label
my_face_dict_n = {"P0": "Elvis",
                  "P1": "Fausto",
                  "P2": "Ale",
                  "P3": "Jenifer",
                  "P4": "Malki",
                  "P5": "Maru",
                  "P6": "Nayta",
                  "P7": "Roberto",
                  "P8": "Samy",
                  "P9": "Sulay",
                    }



In [ ]:
# to show image with label.
def draw_test(name, pred, im):
    myface = my_face_dict[str(pred)]
    BLACK = [0, 0, 0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100, cv2.BORDER_CONSTANT, value=BLACK)
    cv2.putText(expanded_image, myface, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('test', expanded_image)

# select random images from testing folder, returns path to image.
def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0, len(folders))
    path_class = folders[random_directory]
    print("Class - " + my_face_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)

In [ ]:
# lets test with 50 random images to check model.tflite
for i in range(0, 50):
    input_im = getRandomImage("/Users/samcn96/PycharmProjects/training/rostros/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

    # check image size and resize if needed
    input_im = cv2.resize(input_im, (224, 224), interpolation=cv2.INTER_LINEAR)
    input_im = input_im / 255
    input_im = input_im.reshape(1, 224, 224, 3)

    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path="mobile/model.tflite")
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test model on random input data.
    input_shape = input_details[0]['shape']

    input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    
    # predict on input_image
    interpreter.set_tensor(input_details[0]['index'], input_im.astype(np.float32))

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    res = np.argmax(output_data, axis=1)
    
    # print class prediction
    print(res)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original)
    cv2.waitKey(0)

cv2.destroyAllWindows()